In [1]:
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.schema import BaseChatMessageHistory
from langchain_aws import ChatBedrock

from langchain_core.runnables.history import RunnableWithMessageHistory
import time
from botocore.exceptions import ClientError

In [2]:
def insert_chat_message(session_id, message_content, role, timestamp=None):
    """
    Insert a chat message into the DynamoDB table.
    
    Args:
        session_id (str): The session identifier
        message_content (str): The content of the message
        role (str): The role of the message sender (e.g., 'human', 'ai')
        timestamp (str, optional): Message timestamp. Defaults to current time if None.
    
    Returns:
        dict: The response from DynamoDB
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    # Use current time if timestamp not provided
    if timestamp is None:
        sequence = int(time.time() * 1000)  # millisecond timestamp
        timestamp = datetime.utcnow().isoformat()
    else:
        # Convert timestamp to sequence number
        dt = datetime.fromisoformat(timestamp)
        sequence = int(dt.timestamp() * 1000)
    
    # Prepare item to insert
    item = {
        'session_id': session_id,
        'sequence': sequence,
        'role': role,
        'message': message_content,
        'timestamp': timestamp
    }
    
    try:
        # Insert the item into the table
        response = table.put_item(Item=item)
        print(f"Message inserted successfully: {sequence}")
        return response
    except ClientError as e:
        print(f"Error inserting message: {e.response['Error']['Message']}")
        raise

In [3]:
def get_chat_messages(session_id, ascending=True, limit=None):
    """
    Retrieve chat messages for a session, sorted by sequence.
    
    Args:
        session_id (str): The session identifier
        ascending (bool): Sort order - True for ascending (oldest first), False for descending (newest first)
        limit (int, optional): Maximum number of messages to retrieve
    
    Returns:
        list: List of message items
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    messages = []
    
    try:
        # Prepare query parameters
        query_params = {
            'KeyConditionExpression': Key('session_id').eq(session_id),
            'ScanIndexForward': ascending  # True for ascending, False for descending
        }
        
        # Add limit if specified
        if limit:
            query_params['Limit'] = limit
        
        # Query messages for the session, sorted by sequence
        response = table.query(**query_params)
        
        # Add results to our messages list
        messages.extend(response['Items'])
        
        # Handle pagination if there are more results and no limit specified
        while 'LastEvaluatedKey' in response and not limit:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ScanIndexForward=ascending,
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            messages.extend(response['Items'])
        
        print(f"Found {len(messages)} messages for session {session_id}")
        return messages
        
    except ClientError as e:
        print(f"Error retrieving messages: {e.response['Error']['Message']}")
        return []

In [4]:
def delete_all_session_messages(session_id):
    """
    Delete all messages for a specific session.
    
    Args:
        session_id (str): The session identifier
    
    Returns:
        dict: Summary of deletion operation
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    try:
        # First, query to get all messages for the session
        response = table.query(
            KeyConditionExpression=Key('session_id').eq(session_id)
        )
        
        items = response['Items']
        
        # Handle pagination if there are more results
        while 'LastEvaluatedKey' in response:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            items.extend(response['Items'])
        
        # If no items found, return early
        if not items:
            print(f"No messages found for session {session_id}")
            return {"deleted_count": 0}
        
        # Delete each item
        deleted_count = 0
        for item in items:
            table.delete_item(
                Key={
                    'session_id': session_id,
                    'sequence': item['sequence']
                }
            )
            deleted_count += 1
        
        print(f"Deleted {deleted_count} messages for session {session_id}")
        return {"deleted_count": deleted_count}
        
    except ClientError as e:
        print(f"Error deleting messages: {e.response['Error']['Message']}")
        raise

In [5]:
class DynamoDBHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, table_name: str = "conversations-table"):
        self.session_id = session_id
        self.table_name = table_name
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(table_name)
    
    @property
    def messages(self) -> list[BaseMessage]:
        """DynamoDB에서 메시지 로드"""
        try:
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id),
                ScanIndexForward=True,  
                Limit=10,
            )
            
            messages = []
            for item in response['Items']:
                if item['role'] == 'human':
                    messages.append(HumanMessage(content=item['message']))
                elif item['role'] == 'ai':
                    messages.append(AIMessage(content=item['message']))
            
            return messages
        except Exception as e:
            print(f"메시지 로드 실패: {e}")
            return []
    
    def add_message(self, message: BaseMessage) -> None:
        """새 메시지를 DynamoDB에 저장"""
        sequence = int(time.time() * 1000)  # millisecond timestamp
        
        if isinstance(message, HumanMessage):
            role = 'human'
        elif isinstance(message, AIMessage):
            role = 'ai'
        else:
            return
        
        self.table.put_item(
            Item={
                'session_id': self.session_id,
                'sequence': sequence,
                'role': role,
                'message': message.content,
                'timestamp': datetime.now().isoformat()
            }
        )
    
    def clear(self) -> None:
        """대화 기록 삭제"""
        try:
            # Query all items for this session
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id)
            )
            
            # Delete each item
            for item in response['Items']:
                self.table.delete_item(
                    Key={
                        'session_id': item['session_id'],
                        'sequence': item['sequence']
                    }
                )
        except Exception as e:
            print(f"메시지 삭제 실패: {e}")

In [6]:
response1 = insert_chat_message(
    session_id="test-session-001",
    message_content="안녕하세요, 도움이 필요합니다.",
    role="human"
)

Message inserted successfully: 1762143127701


In [7]:
result=get_chat_messages(session_id="test-session-001")

Found 9 messages for session test-session-001


In [8]:
result

[{'message': '안녕하세요',
  'role': 'human',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-03T03:55:54.920898',
  'sequence': Decimal('1762142154920')},
 {'message': '안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어와 영어 모두 편하게 사용하실 수 있으니, 어떤 언어든 편하신 대로 대화하시면 됩니다. 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든 말씀해 주세요!\n\n혹시 특별히 관심 있는 주제나 이야기하고 싶은 것이 있으신가요?',
  'role': 'ai',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-03T03:55:54.982191',
  'sequence': Decimal('1762142154982')},
 {'message': '상사병은 무엇인가요?',
  'role': 'human',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-03T03:56:05.389415',
  'sequence': Decimal('1762142165389')},
 {'message': '안녕하세요! 상사병에 대해 설명해드릴게요! 😊\n\n상사병(相思病)은 한자어로 "서로 그리워하는 병"이라는 뜻으로, 사랑하는 사람을 그리워하며 생기는 마음의 병을 의미합니다. \n\n**상사병의 특징들:**\n\n🔹 **증상들**\n- 사랑하는 사람이 자꾸 생각나서 집중이 안 됨\n- 식욕 부진이나 잠을 제대로 못 잠\n- 가슴이 답답하고 한숨이 자주 나옴\n- 멍하니 있는 시간이 많아짐\n- 그 사람과 관련된 것들에 과도하게 민감해짐\n\n🔹 **문학적 표현**\n- 고전 문학에서 자주 등장하는

In [21]:
delete_all_session_messages("test-session-001")

Deleted 8 messages for session test-session-001


{'deleted_count': 8}

In [10]:
class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None

In [11]:
llm = AdvancedLLM().llm

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

# Your prompt template (already correct)
template_dynamo = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

Relevant Information:

{chat_history}

Conversation:
Human: {question}
AI:"""

prompt_dynamo = PromptTemplate(
    input_variables=["chat_history", "question"], 
    template=template_dynamo
)

# Create chain with DynamoDB history
chain = prompt_dynamo | llm | StrOutputParser()




In [13]:
chain_dynamo = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: DynamoDBHistory(session_id, "conversations-table"),
    input_messages_key="question",
    history_messages_key="chat_history"
)

# Usage
response = chain_dynamo.invoke(
    {"question": "안녕하세요"},
    config={"configurable": {"session_id": "test-session-001"}}
)



In [14]:
response

'안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어와 영어 모두 편하게 사용하실 수 있으니, 어떤 언어로든 대화하시면 됩니다. 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든 말씀해 주세요. 함께 즐거운 대화를 나눠보아요!'

In [15]:
res1=chain_dynamo.invoke( {"question": "상사병은 무엇인가요?"},
    config={"configurable": {"session_id": "test-session-001"}})

In [16]:
res1

'안녕하세요! 상사병에 대해 설명해드릴게요! 😊\n\n상사병(相思病)은 한자어로 "서로 그리워하는 병"이라는 뜻으로, 사랑하는 사람을 그리워하며 생기는 마음의 병을 의미합니다. \n\n**상사병의 특징들:**\n\n🎭 **문학적 표현**: 주로 고전문학이나 시에서 사용되는 낭만적인 표현이에요\n- 조선시대 문학작품에서 자주 등장하는 개념입니다\n- "춘향전"에서 이몽룡과 춘향이가 겪는 감정이 대표적인 예죠\n\n💔 **증상들**:\n- 사랑하는 사람을 만나지 못해 생기는 그리움\n- 식욕부진, 잠 못 이루기\n- 멍하니 있거나 한숨 자주 쉬기\n- 집중력 저하\n\n🔬 **현대적 관점**:\n- 의학적으로는 실제 질병이 아니라 감정적 상태를 표현한 것\n- 현재는 "짝사랑의 고통" 정도로 이해됩니다\n- 심한 경우 우울감이나 스트레스로 이어질 수 있어요\n\n재미있게도 영어권에서는 "lovesickness"라고 비슷한 표현이 있답니다! 혹시 상사병과 관련해서 더 궁금한 점이 있으신가요?'

In [17]:
res2=chain_dynamo.invoke( {"question": "나는 상사병이 상사로 인해 스트레스 받아서 생기는 마음의 병이라고 나는생각해"},
    config={"configurable": {"session_id": "test-session-001"}})

In [18]:
res2

'아하! 정말 재미있는 해석이네요! 😄\n\n말씀하신 것처럼 "상사병"을 현대적으로 해석하면 정말 그럴 수 있겠어요! \n\n**현대인의 "상사병" 해석:**\n\n👔 **직장에서의 상사병**:\n- 까다로운 상사 때문에 받는 스트레스\n- 월요병보다 더 심각한 "상사 때문에 생기는 병"\n- 상사 얼굴만 봐도 속이 쓰리고 머리가 아픈 증상들\n\n😅 **증상들**:\n- 출근길에 한숨 늘어나기\n- 상사 이름만 들어도 혈압 상승\n- 주말에도 상사 생각에 스트레스 받기\n- "사표 쓰고 싶다"는 생각 반복\n\n사실 한자로는 相思病(서로 그리워하는 병)이지만, 한국어로 들으면 정말 "상사(上司) + 병"으로 들릴 수 있어요! 특히 직장생활 하시는 분들에게는 훨씬 현실적이고 공감되는 해석이겠네요! \n\n혹시 직장에서 힘든 상사 때문에 스트레스 받고 계신 건 아니시죠? 😊 요즘 직장인들에게는 오히려 이 해석이 더 와닿을 것 같아요!'

In [19]:
res3=chain_dynamo.invoke( {"question": "내가 생각하는 상사병은 무엇인가요?"},
    config={"configurable": {"session_id": "test-session-001"}})

In [20]:
res3

'앞서 대화에서 말씀해주신 내용을 보면, 당신이 생각하는 상사병은:\n\n**"상사로 인해 스트레스 받아서 생기는 마음의 병"** 이라고 하셨어요! 😊\n\n즉, 전통적인 相思病(서로 그리워하는 병)이 아니라, 현대적 해석으로 **상사(上司) + 병**의 개념으로 이해하고 계시는 거죠!\n\n**당신이 생각하시는 상사병의 특징:**\n- 직장에서 까다로운 상사 때문에 받는 스트레스\n- 상사와의 관계에서 오는 정신적, 육체적 고통\n- 현대 직장인들이 실제로 겪는 현실적인 문제\n\n정말 창의적이고 현실적인 해석이라고 생각해요! 한자어 "상사병"을 현대 한국어 "상사병"으로 재해석하신 거네요. 사실 많은 직장인들에게는 이 해석이 훨씬 더 와닿고 공감될 것 같아요! \n\n언어유희적으로도 재미있고, 현실적으로도 매우 적절한 해석이라고 생각합니다! 👍'